In [6]:
from google.cloud import bigquery

client = bigquery.Client()

In [7]:
# --- Para a consulta CREATE MODEL (VERSÃO FINAL E ROBUSTA) ---

print("Iniciando o treinamento do modelo no BigQuery ML (versão final)...")

create_model_query = """
CREATE OR REPLACE MODEL `analytics.modelo_engajamento_spa`
OPTIONS (
  MODEL_TYPE = 'KMEANS',
  NUM_CLUSTERS = 4,
  KMEANS_INIT_METHOD = 'KMEANS++',
  STANDARDIZE_FEATURES = TRUE -- O escalonamento continua sendo feito automaticamente
) AS
WITH
  source_data AS (
    -- Passo 1: Pega os dados brutos da sua função (sem alteração)
    SELECT
      eng.postsAmount,
      eng.postRepliesAmount,
      eng.helpRequestsAmount,
      eng.helpRequestRepliesAmount,
      eng.performance_rate
    FROM
      analytics.fn_engajamento (
        DATE('2020-01-01'), DATE('2021-12-31'), TRUE, 'redu-digital',
        NULL, NULL, NULL, NULL, NULL, 'SPA', 'QUARTER'
      ) AS eng
    INNER JOIN
      replicas.user_environment_associations AS filtro
      ON eng.cliente = filtro.cliente AND eng.user_id = filtro.user_id
    WHERE
      filtro.role NOT IN ('teacher', 'tutor', 'environment_admin')
  ),
  data_with_logs AS (
    -- Passo 2: Aplica a transformação logarítmica (np.log1p)
    SELECT
      LOG(postsAmount + 1) AS postsAmount_log,
      LOG(postRepliesAmount + 1) AS postRepliesAmount_log,
      LOG(helpRequestsAmount + 1) AS helpRequestsAmount_log,
      LOG(helpRequestRepliesAmount + 1) AS helpRequestRepliesAmount_log,
      LOG(performance_rate + 1) AS performance_rate_log
    FROM
      source_data
  ),
  imputed_data AS (
    -- Passo 3: Imputa os valores nulos usando a média da coluna de log.
    -- Esta é a etapa que não podia ser feita dentro do TRANSFORM.
    SELECT
      COALESCE(postsAmount_log, AVG(postsAmount_log) OVER ()) AS postsAmount,
      COALESCE(postRepliesAmount_log, AVG(postRepliesAmount_log) OVER ()) AS postRepliesAmount,
      COALESCE(helpRequestsAmount_log, AVG(helpRequestsAmount_log) OVER ()) AS helpRequestsAmount,
      COALESCE(helpRequestRepliesAmount_log, AVG(helpRequestRepliesAmount_log) OVER ()) AS helpRequestRepliesAmount,
      COALESCE(performance_rate_log, AVG(performance_rate_log) OVER ()) AS performance_rate
    FROM
      data_with_logs
  )
-- Passo 4: O SELECT final para o modelo é simples. Ele apenas consome
-- os dados já totalmente pré-processados da CTE 'imputed_data'.
-- Nenhuma cláusula TRANSFORM é necessária aqui.
SELECT
  *
FROM
  imputed_data;
"""

# O restante do seu código para executar o job permanece o mesmo
job = client.query(create_model_query)
job.result()  # Aguarda a conclusão do treinamento

print(f"Modelo `analytics.modelo_engajamento_spa` criado ou substituído com sucesso!")

Iniciando o treinamento do modelo no BigQuery ML (versão final)...
Modelo `analytics.modelo_engajamento_spa` criado ou substituído com sucesso!


In [11]:
import pandas as pd
import numpy as np

# Certifique-se de que o 'client' do BigQuery ainda está ativo na sua sessão.

# NOTA: Substitua 'seu_projeto.analytics.modelo_engajamento_spa' se o nome do seu modelo for diferente.
model_name = 'analytics.modelo_engajamento_spa'

# --- Passo 1: Buscar os centroides PADRONIZADOS (valores Z-score) ---
print("Buscando os centroides padronizados...")
centroids_std_query = f"""
SELECT
  centroid_id,
  feature,
  numerical_value AS standardized_value
FROM
  ML.CENTROIDS(MODEL `{model_name}`)
ORDER BY
  centroid_id, feature;
"""
df_centroids_std = client.query(centroids_std_query).to_dataframe()


# --- Passo 2: Buscar as informações das features (média e desvio padrão usados no treino) ---
print("Buscando as estatísticas de escalonamento (média e desvio padrão)...")
feature_info_query = f"""
SELECT
  input AS feature,
  mean,
  stddev
FROM
  ML.FEATURE_INFO(MODEL `{model_name}`);
"""
df_feature_info = client.query(feature_info_query).to_dataframe()


# --- Passo 3: Despadronizar os centroides manualmente no Pandas ---
print("Realizando a despadronização no notebook...")

# Juntar as duas tabelas pela coluna 'feature'
df_merged = pd.merge(df_centroids_std, df_feature_info, on='feature')

# Aplicar a fórmula inversa do Z-score: valor_original = (valor_padronizado * desvio_padrão) + média
df_merged['destandardized_value'] = (df_merged['standardized_value'] * df_merged['stddev']) + df_merged['mean']

# Pivotar a tabela para o formato final: um cluster por linha, uma feature por coluna
df_centroids = df_merged.pivot(
    index='centroid_id',
    columns='feature',
    values='destandardized_value'
)

# Reordenar as colunas para a mesma ordem do seu treino original
feature_order = [
    'postsAmount', 'postRepliesAmount', 'helpRequestsAmount',
    'helpRequestRepliesAmount', 'performance_rate'
]
df_centroids = df_centroids[feature_order]


# --- Exibe o DataFrame com as médias de cada variável por cluster ---
print("\n--- Médias das Variáveis (em escala logarítmica) por Cluster ---")
display(df_centroids)


# --- Opcional: Revertendo o Log para ver as médias na escala original (aproximada) ---
df_centroids_original_scale = df_centroids.apply(np.expm1)

print("\n--- Médias das Variáveis (em escala original aproximada) por Cluster ---")
display(df_centroids_original_scale)

Buscando os centroides padronizados...
Buscando as estatísticas de escalonamento (média e desvio padrão)...
Realizando a despadronização no notebook...

--- Médias das Variáveis (em escala logarítmica) por Cluster ---


feature,postsAmount,postRepliesAmount,helpRequestsAmount,helpRequestRepliesAmount,performance_rate
centroid_id,,,,,
1,0.006122,0.000597,0.000081,0.000067,0.007167
2,0.134599,0.006802,0.005912,0.000169,0.012174
3,0.076558,0.015637,0.001579,0.006482,0.010442
4,0.106338,0.029921,0.000081,0.000067,0.011853



--- Médias das Variáveis (em escala original aproximada) por Cluster ---


feature,postsAmount,postRepliesAmount,helpRequestsAmount,helpRequestRepliesAmount,performance_rate
centroid_id,,,,,
1,0.006141,0.000597,0.000081,0.000067,0.007193
2,0.144078,0.006825,0.005930,0.000169,0.012248
3,0.079565,0.015760,0.001580,0.006503,0.010496
4,0.112198,0.030374,0.000081,0.000067,0.011923
